# Part 3A - Planning & Preparing Data

- 01/26/24

- NEED TO PREPARE/PLAN FOR FULL APP WITH TABULAR DATA + REVIEWS + MANUSCRIPTS.

### Prep TMDB API Data (Was Split Across Several Notebooks)

In [1]:
## Importing custom function for project
!pip install -U dojo_ds -q 
import dojo_ds as fn
# %load_ext autoreload
# %autoreload 2
# import custom_functions as fn


import os,glob,json
from pprint import pprint
import pandas as pd
pd.set_option("display.max_columns",200)


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# !pip install -U dojo_ds -q
# import dojo_ds as ds
# ds.__version__

## Setting Filepaths Config Files

>- Filepath to-dos:
>    - Add names of model insights figures to FPATHS     

In [9]:
import json, os
from pprint import pprint

# Define filename for project config filepaths json file
FPATHS_FILE = "config/filepaths.json"
os.makedirs(os.path.dirname(FPATHS_FILE), exist_ok=True)

# Define Filepaths
FPATHS = dict(
    data={
        "raw": {

            # TO DO: ADD THE DIRS FOR THE SPLIT REVIEWS AND UPDATE DATA/01_ Notebook
            'reviews-subset_most-commonly-reviewed_csv':'data/subset/amazon-reviews-subset-grocery-most-common-products.csv.gz',
            # Brand subset
            "reviews-subset_selected-brand_csv":"data/subset/amazon-reviews-subset-brand-Miracle Noodle.csv.gz",
            "metadata_csv":'data/subset/amazon-metadata-subset-grocery-most-common-products.csv.gz',
            # Movie Reviews (no movie data or categories
            # "movie-reviews-tmdb_csv": "Data-NLP/tmdb-movie-reviews.csv.gz",
            # TMDB Movie Info to Use for...NLP? (Why save this separate? )
            # "cleaned-tmdb-movie-info-tmdb_csv": "Data-NLP/tmdb-movie-info.csv.gz", 
            # Combined Movie Reviews with financial data
            # "reviews-with-movie-info_json": "Data-NLP/combined-tmdb-movie-reviews-with-info.json", 
            # "movie-reviews-with-info_json":
            # ...??
            # "eda": "Data-NLP/eda-movie-reviews.csv.gz",
        },
        "processed":{
            # "processed-reviews-spacy_csv":"data/processed/processed-reviews.csv",
            "processed-reviews-spacy_json":"data/processed/processed-reviews.json",
            "processed-reviews-spacy_joblib": "data/processed/processed-reviews.joblib",
            # "processed-reviews-spacy_json": "Data-NLP/processed-nlp-reviews.json",
        },
        "app": {
            # Version of tabular data to load into streamlit app (unless decide to do AWS)
            # "movie-data_csv": "app-assets/movie-data-streamlit.csv", 
            
            # "movie-reviews": ???:
                # Same as raw>movie-reviews-with-info-json?
                # or Same as ml>'reviews-with-target-json'?? 
        },
        "ml-nlp": {
            # Final dataframe of machine learning for NLP ( Use Data-NLP Folder for these models)
            "reviews-with-target_json": "data/modeling/processed-nlp-reviews-for-ml.json",
            "train_joblib": "data/modeling/training-data.joblib",  # (X_train,y_train)
            "test_joblib": "Data-NLP/modeling/testing-data.joblib",  # (X_test,y_test)
        },
        "ml-tabular": {
            # Final dataframe of machine learning ( Use Data Folder for these models)
            "reviews-with-ml-target_json": "Data/modeling/processed-movie-data-for-ml.json", # "Data-NLP/modeling/processed-nlp-reviews-for-ml.json",
            "train_joblib": "data/modeling/training-data.joblib",  # (X_train,y_train)
            "test_joblib": "data/modeling/testing-data.joblib",  # (X_test,y_test)
        },
        "nn": {
            "train_dir": "data/modeling/training-data-tf/",  # train_ds
            "test_dir": "data/modeling/testing-data-tf/",  # test_ds
        },
    },
    images={
        "banner": "images/app-banner.png",
    },
    # # Additional metadata (target lookup,etc.)
    metadata={
        "target_lookup": "data/metadata/target-lookup.json",
    },
    # Any images to be displayed in the app
    eda={
        "wordclouds-by-group_png": "images/wordclouds-compare-groups.png",
        # "wordclouds-by-rating_png": "images/wordclouds-compare-rating.png",
        "scattertext-by-group_html": "app-assets/scattertext-roi.html",
        # "scattertext-by-rating_html": "app-assets/scattertext-rating.html",

    },
    models={
        # Machine Learning Models and results
        "ml": {
            "bayes_joblib": "models/bayes-clf.joblib",
            "random_forest_joblib": "models/random-forest.joblib",
            "logreg_joblib": "models/log-reg.joblib",
        },
        # Neural networks and results
        "nn": {
            "LSTM_dir": "models/keras/lstm/",
            "GRU_dir": "models/keras/gru/",
            "Attention_model_dir": "models/keras/attn/",
        },
    },
)
# Use fn for local package, ds for pip version
fn.utils.create_directories_from_paths(FPATHS)
# ds.utils.create_directories_from_paths(FPATHS)

print('[i] FPATHS Dictionary:\n')
pprint(FPATHS)

## Save the filepaths
with open(FPATHS_FILE, "w") as f:
    json.dump(FPATHS, f)
    print(f"\n[i] Saved FPATHS to {FPATHS_FILE}")

[i] FPATHS Dictionary:

{'data': {'app': {},
          'ml-nlp': {'reviews-with-target_json': 'data/modeling/processed-nlp-reviews-for-ml.json',
                     'test_joblib': 'Data-NLP/modeling/testing-data.joblib',
                     'train_joblib': 'data/modeling/training-data.joblib'},
          'ml-tabular': {'reviews-with-ml-target_json': 'Data/modeling/processed-movie-data-for-ml.json',
                         'test_joblib': 'data/modeling/testing-data.joblib',
                         'train_joblib': 'data/modeling/training-data.joblib'},
          'nn': {'test_dir': 'data/modeling/testing-data-tf/',
                 'train_dir': 'data/modeling/training-data-tf/'},
          'processed': {'processed-reviews-spacy_joblib': 'data/processed/processed-reviews.joblib',
                        'processed-reviews-spacy_json': 'data/processed/processed-reviews.json'},
          'raw': {'metadata_csv': 'data/subset/amazon-metadata-subset-grocery-most-common-products.csv.gz',
   

In [10]:
with open(FPATHS_FILE) as f:
    TEST = json.load(f)
# pprint(TEST)
TEST

{'data': {'raw': {'reviews-subset_most-commonly-reviewed_csv': 'data/subset/amazon-reviews-subset-grocery-most-common-products.csv.gz',
   'reviews-subset_selected-brand_csv': 'data/subset/amazon-reviews-subset-brand-Miracle Noodle.csv.gz',
   'metadata_csv': 'data/subset/amazon-metadata-subset-grocery-most-common-products.csv.gz'},
  'processed': {'processed-reviews-spacy_json': 'data/processed/processed-reviews.json',
   'processed-reviews-spacy_joblib': 'data/processed/processed-reviews.joblib'},
  'app': {},
  'ml-nlp': {'reviews-with-target_json': 'data/modeling/processed-nlp-reviews-for-ml.json',
   'train_joblib': 'data/modeling/training-data.joblib',
   'test_joblib': 'Data-NLP/modeling/testing-data.joblib'},
  'ml-tabular': {'reviews-with-ml-target_json': 'Data/modeling/processed-movie-data-for-ml.json',
   'train_joblib': 'data/modeling/training-data.joblib',
   'test_joblib': 'data/modeling/testing-data.joblib'},
  'nn': {'train_dir': 'data/modeling/training-data-tf/',
   't

In [7]:
TEST['data'].keys()

dict_keys(['raw', 'processed', 'app', 'ml-nlp', 'ml-tabular', 'nn'])